In [ ]:
! pip install ta


In [ ]:
import tensorflow as tf
from keras.layers import Lambda
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Input, Conv1D, LSTM, Dense, Reshape, Lambda, Dropout,BatchNormalization
import keras.backend as K
import ta

btc_data = yf.download('BTC-USD', period='1y', interval='1d')

candlestick_data = btc_data[['Open', 'High', 'Low', 'Close']]

btc_data['RSI'] = ta.momentum.RSIIndicator(btc_data['Close']).rsi()
btc_data['MACD'] = ta.trend.MACD(btc_data['Close']).macd()

full_data = btc_data[['Open', 'High', 'Low', 'Close', 'RSI', 'MACD']].dropna()


scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(full_data)

In [ ]:
def create_dataset(data, time_step=60, forecast_step=30):
    X, y = [], []
    for i in range(len(data) - time_step - forecast_step):
        X.append(data[i:(i + time_step), :])
        y.append(data[(i + time_step):(i + time_step + forecast_step), :])
    return np.array(X), np.array(y)
time_step = 60
forecast_step = 30
X, y = create_dataset(scaled_data, time_step, forecast_step)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [ ]:
model = Sequential()
model.add(Input(shape=(time_step, X.shape[2])))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))

model.add(LSTM(100, return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(100, return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(100, return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(50, return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(50, return_sequences=False, recurrent_dropout=0.2))

model.add(Lambda(lambda x: tf.stack([
    tf.abs(x[..., 3] - x[..., 0]) /
    tf.maximum(x[..., 3], x[..., 0]),  # درصد بدنه کندل (Close - Open)
    tf.abs(x[..., 1] - x[..., 2]) /
    tf.maximum(x[..., 1], x[..., 2])   # درصد High و Low (High - Low)
], axis=-1)))

model.add(Dense(forecast_step * X.shape[2] ))
model.add(Reshape((forecast_step, X.shape[2])))
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=200, batch_size=64)


In [ ]:
loss = model.evaluate(X_test, y_test)

In [ ]:
last_sequence = scaled_data[-time_step:]  # آخرین 60 تایم‌استپ از داده‌ها

# تغییر شکل داده‌ها برای ورودی به مدل (براساس تعداد ویژگی‌ها)
last_sequence = last_sequence.reshape((1, time_step, X.shape[2]))  # تنظیم تعداد ویژگی‌ها

predicted_data = model.predict(last_sequence)  # پیش‌بینی داده‌ها

# تبدیل خروجی به پیش‌بینی برای 30 روز آینده (براساس ویژگی‌های انتخاب شده)
predicted_data = predicted_data.reshape(-1, X.shape[2])  # بازسازی داده‌ها با تعداد ویژگی‌ها
predicted_data = scaler.inverse_transform(predicted_data)  # بازگرداندن داده‌ها به مقیاس اصلی
predicted_candlesticks = predicted_data[:, :4]


In [ ]:
!pip install mplfinance

In [ ]:
# محاسبه درصد بدنه کندل‌ها
percentage_bodies = np.abs(predicted_candlesticks[:, 3] - predicted_candlesticks[:, 0]) / np.maximum(
    predicted_candlesticks[:, 3], predicted_candlesticks[:, 0])

percentage_high_low = np.abs(predicted_candlesticks[:, 1] - predicted_candlesticks[:, 2]) / np.maximum(
    predicted_candlesticks[:, 1], predicted_candlesticks[:, 2])
# تبدیل numpy array به pandas DataFrame
df_predicted = pd.DataFrame(predicted_candlesticks, columns=['Open', 'High', 'Low', 'Close'])

# اضافه کردن درصد بدنه و درصد High-Low به DataFrame
df_predicted['Body Percentage'] = percentage_bodies
df_predicted['High-Low Percentage'] = percentage_high_low

# ایجاد ستون تاریخ برای 30 روز آینده
df_predicted['Date'] = pd.date_range(start='2024-10-05', periods=len(df_predicted), freq='D')

# تنظیم ستون 'Date' به عنوان ایندکس DataFrame
df_predicted.set_index('Date', inplace=True)

In [ ]:
df_predicted.to_csv('predicted_candlesticks.csv')

print("داده‌های پیش‌بینی شده در فایل 'predicted_candlesticks.csv' ذخیره شد.")


In [ ]:
# نمایش کندل‌های پیش‌بینی شده برای 30 روز آینده
print("Predicted Candlesticks for the Next 30 Days:")

for i, candlestick in enumerate(predicted_candlesticks):
    open_price = candlestick[0]
    high_price = candlestick[1]
    low_price = candlestick[2]
    close_price = candlestick[3]

    # محاسبه درصد بدنه کندل
    body_percentage = abs(close_price - open_price) / max(close_price, open_price)

    # محاسبه درصد سایه بالا
    upper_shadow_percentage = (high_price - max(open_price, close_price)) / high_price

    # محاسبه درصد سایه پایین
    lower_shadow_percentage = (min(open_price, close_price) - low_price) / low_price

    # نمایش اطلاعات کندل
    print(f"Day {i+1}: Open: {open_price:.2f}, High: {high_price:.2f}, Low: {low_price:.2f}, Close: {close_price:.2f}, "
          f"Body Percentage: {body_percentage:.4f}, Upper Shadow Percentage: {upper_shadow_percentage:.4f}, Lower Shadow Percentage: {lower_shadow_percentage:.4f}")


In [ ]:
# نمایش پیش‌بینی کندل‌های 30 روز آینده
print("Predicted Candlesticks for the Next 30 Days:")
for i, candlestick in enumerate(predicted_data[:, :4]):  # فقط 4 ستون اول (Open, High, Low, Close)
    print(
        f"Day {i+1}: Open: {candlestick[0]}, High: {candlestick[1]}, Low: {candlestick[2]}, Close: {candlestick[3]}"
    )

# خواندن داده‌ها از فایل CSV
df = pd.read_csv('predicted_candlesticks.csv')

# اطمینان از تبدیل صحیح داده‌ها
df['Date'] = pd.to_datetime(df['Date'])

# انتخاب ستون‌های مرتبط با نمودار کندل (Open, High, Low, Close)
df_candlestick = df[['Date', 'Open', 'High', 'Low', 'Close']]

# تنظیم ستون 'Date' به عنوان ایندکس برای رسم نمودار
df_candlestick.set_index('Date', inplace=True)

# ادامه کد رسم نمودار کندل
import mplfinance as mpf
mpf.plot(df_candlestick, type='candle', style='charles',
         title='Predicted Candlesticks for the Next 30 Days', volume=False)
